In [16]:
import xml.etree.ElementTree as ET
def load_payments_from_XML_sber(file_name):
    # возвращает словарь результатов:
    # df: DataFrame
    # Параметры выписки (d1, d2, sum1, sum2, приходы, расходы, метка карты)

    res = {}
    res["error_text"] = "" # значение поумолчанию, т.е. нет ошибок
    res['success'] = False
    res['prop'] = [0,0,0,0]  # res['prop'] [Остаток1, Остаток2, Расходы, Приходы]

    # tag_root
    xlm = ET.parse(file_name)
    tag_root = xlm.getroot()

    level =   '{urn:schemas-microsoft-com:office:spreadsheet}Worksheet'
    level += '/{urn:schemas-microsoft-com:office:spreadsheet}Table'
    level += '/{urn:schemas-microsoft-com:office:spreadsheet}Row'

    ind = 0
    rows = []
    rows1 = []

    # считать данные в список
    for tag_row in tag_root.findall(level):
        row = []
        for i_cel, tag in enumerate(tag_row.findall('{urn:schemas-microsoft-com:office:spreadsheet}Cell')):            
            text = ''
            for t in tag.itertext():
                text += t
            row.append(text)
        rows.append(row)
        ind +=1
    
    ind = 19
    n_rows = len(rows)
    while ind < n_rows: 

        row=rows[ind]

        # все строки, длиной 8 ячеек
        #   Date	    Time  ...   Name	...	         Amount	....	Balance	Amount_num	Balance_num
        # 0	2022.03.31	15:44		Перевод с карты		5 000,00		337.94	5000.00	337.94
        # 1	2022.03.31	202880		SBOL перевод 2202****6497 В. СЕРГЕЙ СЕРГЕЕВИЧ					NaN	NaN
        if len(row)==8:
            rows1.append(row) 
        # т.к. дина ячеек <> 8
        elif len(row) == 1:       
            if rows[ind-1] == ['В валюте счёта']:
                
                spaces = '     '
                while (spaces in row[0]) and (n_rows-ind > 5):
                        # проверить задовение 2-х строк в одну (если точнее, то 5 строк в 10. цветной пример см. в xlsx февраль 2022, "Maestro •••• 6632")
                        packet=1     # число строк в группе (задовения)
                        if spaces in rows[ind+2*packet][0]: 
                            packet=2
                            if spaces in rows[ind+2*packet][0]: 
                                packet=3
                                if spaces in rows[ind+2*packet][0]: 
                                    # перебор. делаем ошибку. 
                                    res["error_text"] = f'Ошибка формата Сбер_XML. Превышен допустимый размер пакета строк (код 311)'
                                    return res
                        for p_ind in range(0, packet):                                

                            # считываем строки и записываем их в rows1 (длиной 8)
                            r1=8*[""]
                            r2=8*[""]

                            # val = ['21.02.2022         13:54']     
                            row = rows[ind+2*p_ind + 0]
                            val = row[0]  
                            v1 = val[0:10]
                            v2 = val[-5:]
                            r1[0] = v1
                            r1[1] = v2

                            # val = ['22.02.2022          209281']
                            row = rows[ind+2*p_ind + 1]
                            val = row[0]
                            v1 = val[0:10]
                            v2 = val[-6:]
                            r2[0] = v1
                            r2[1] = v2

                            # val = ['Перевод с карты']
                            row = rows[ind+2*p_ind + 2]
                            val = row[0]
                            r1[3] = val

                            # val = ['SBOL перевод 4276****9099 П. ДЕНИС ЮРЬЕВИЧ']
                            row = rows[ind+5*p_ind + 3]
                            val = row[0]
                            r2[3] = val

                            # val = ['125 000,00                           70 228,77']
                            row = rows[ind+2*p_ind + 4]
                            val = row[0]
                            lst = val.split('   ')
                            v1 = lst[0]
                            v2 = lst[-1]
                            r1[5] = v1
                            r1[7] = v2
                            rows1.append(r1)
                            rows1.append(r2)
                        ind += 5*packet
        ind +=1
    return rows1        
            
        
        
file_name_input = 'D:\\YandexDisk\\Мои ДОКУМЕНТЫ\\ООО Арсенал-строй (уу)\\!!! Управленческй учет\\Закрытие 2022-02 (февраль)\\Документ-2022-03-05 Maestro 6632.xml'
rows1 = load_payments_from_XML_sber(file_name_input)



In [15]:
for row in rows1:
    print(row)

['2022-02-27T00:00:00.000', '08:52', '', 'Перевод с карты', '', '5 000,00', '', '3 432,97']
['2022-02-28T00:00:00.000', '299600', '', 'SBOL перевод 2202****0202', '', '', '', '']
['2022-02-26T00:00:00.000', '13:31', '', 'Прочие операции', '', '4 800,00', '', '8 432,97']
['2022-02-26T00:00:00.000', '206591', '', 'RNCB', '', '', '', '']
['2022-02-25T00:00:00.000', '13:23', '', 'Путешествия', '', '6 010,80', '', '13 232,97']
['2022-02-26T00:00:00.000', '204976', '', 'PZD-ONLINE', '', '', '', '']
['2022-02-25T00:00:00.000', '09:17', '', 'Перевод с карты', '', '5 000,00', '', '19 243,77']
['2022-02-25T00:00:00.000', '248418', '', 'SBOL перевод 2202****0202', '', '', '', '']
['2022-02-23T00:00:00.000', '11:59', '', 'Прочие операции', '', '50', '', '24 243,77']
['2022-02-23T00:00:00.000', '284259', '', 'RNCB', '', '', '', '']
['2022-02-23T00:00:00.000', '11:55', '', 'Прочие операции', '', '450', '', '24 293,77']
['2022-02-23T00:00:00.000', '288926', '', 'RNCB', '', '', '', '']
['2022-02-22T00

In [14]:
import os 
import json
import pandas as pd


if True:
    file_name_input = 'D:\\tmp\\Тесты_log\\test.xml'
    file_name_output = os.path.splitext(file_name_input)[0]+ '.json'

    str = f"d {dt.datetime.today().strftime('%Y-%m-%d %H:%M:%S')}  {file_name_input}"
    print(str)

    with open(file_name_output, 'w') as write_file:
        json.dump(str, write_file)
    
    df = pd.DataFrame().columns.to_list
    # print(file_name_output)


d 2022-05-15 09:01:54  D:\tmp\Тесты_log\test.xml 


In [8]:
import datetime as dt, time

t1 = dt.datetime.now()
# t1.timetuple()
sec = time.mktime(t1.timetuple())
t2 = dt.datetime.fromtimestamp(sec)
print(type(t1.timetuple()))
print(t1)
print(t2)
sec


<class 'time.struct_time'>
2022-05-16 08:36:49.706413
2022-05-16 08:36:49


1652679409.0

In [6]:
import pandas as pd
import numpy as np
df = pd.DataFrame(
    data=np.random.randint (
        0, 10, (6,4)),
    columns=["a", "b", "c", "d"])
nmp=df.to_numpy()
print(nmp) 
print(type(nmp))

[[5 4 8 8]
 [6 8 0 6]
 [6 2 3 0]
 [7 0 7 4]
 [1 5 3 0]
 [3 7 9 2]]
<class 'numpy.ndarray'>


In [1]:
def load_my_modules(prj_name=None, lst=None):

    import sys, os

    if prj_name==None:  prj_name = 'project_KnowledgeBase'       
    if lst==None:       lst = [
            '\\my_methods',
            '\\personal_expenses\\modules'
            # '\\qqq,',        
    ]

    # Получить текущую папку
    cwd = os.getcwd()
    prj_len = len(prj_name)
    i1 = cwd.find(prj_name)
    prj_dir = cwd[0:i1+prj_len]
    # d:\IDE_prj\project_KnowledgeBase

    # '\' -> '\\'
    if not prj_dir.find('\\\\'):
        prj_dir = prj_dir.replace('\\', '\\\\')
    
    # Проверить наличие модулей в sys.path_hook
    # Если отсутствют - добавить
    for module_dir in lst:
        module_dir = prj_dir + module_dir
        if not module_dir in sys.path:
            sys.path.insert(0, module_dir)
    return sys.path
load_my_modules()

['d:\\IDE_prj\\project_KnowledgeBase\\personal_expenses\\modules',
 'd:\\IDE_prj\\project_KnowledgeBase\\my_methods',
 'd:\\IDE_prj\\project_KnowledgeBase\\personal_expenses',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39\\DLLs',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39\\lib',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39',
 '',
 'C:\\Users\\Александр\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'C:\\Users\\Александр\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32',
 'C:\\Users\\Александр\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32\\lib',
 'C:\\Users\\Александр\\AppData\\Roaming\\Python\\Python39\\site-packages\\Pythonwin',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages']

In [118]:
import os
path = 'data_input/my_tickets_json'
lst_files = os.listdir(path)
for fn in lst_files:
    save_json_to_database(fn)
    break

In [8]:
import datetime as dt
dt.datetime.today().strftime("%Y.%m.%d")

'2022.05.05'

In [32]:
import json
import pandas as pd
import datetime as dt
from models import *
import os, shutil



def save_json_to_database(file_name, path_complete=None):

    try:
        with open(file_name, encoding='utf-8') as json_file:
            data = json.load(json_file) #, cls= encoding='cp1251').
            head=data[0]
            items = head.pop('items')
            df=pd.DataFrame(items)

            # Проверить наличие чека в базе данных
            dateTime = dt.datetime.fromtimestamp(head['dateTime'])
            sel = FNS_Receipt.select().where(
                        (FNS_Receipt.dateTime == dateTime)
                      & (FNS_Receipt.fiscalSign >= head['fiscalSign']) 
            )
            if len(sel) > 0:
                return 'duplicate'        

            # Запись в БД. Транзакции удаления/записи    
            with db.atomic() as transaction:  # transaction.rollback()    

                try:
                    # записать чек (заголовок)                    
                    # columns=["date", "date_account", "category", "name", "amount", "balance"]
                    rec = head
                    receipt = FNS_Receipt(
                            dateTime                = dt.datetime.fromtimestamp(rec['dateTime']),
                            localDateTime           = dt.datetime.strptime(rec['localDateTime'], '%Y-%m-%dT%H:%M'),
                            totalSum                = rec['totalSum']/100,
                            user                    = rec['user'],
                            retailPlaceAddress      = rec.pop('retailPlaceAddress', ''),
                            userInn                 = rec['userInn'],
                            requestNumber           = rec['requestNumber'],
                            shiftNumber             = rec['shiftNumber'],
                            operator                = rec['operator'],
                            operationType           = rec['operationType'],
                            cashTotalSum            = rec['cashTotalSum'],
                            ecashTotalSum           = rec['ecashTotalSum'],
                            kktRegId                = rec['kktRegId'],
                            fiscalDriveNumber       = rec['fiscalDriveNumber'],
                            fiscalDocumentNumber    = rec['fiscalDocumentNumber'],
                            fiscalSign              = rec['fiscalSign'],
                            nds18                   = rec.pop('nds18', ''),
                            nds10                   = rec.pop('nds10', ''),
                            code                    = rec['code'],
                            fiscalDocumentFormatVer = rec['fiscalDocumentFormatVer'],
                            retailPlace             = rec['retailPlace'],
                            prepaidSum              = rec['prepaidSum'],
                            creditSum               = rec['creditSum'],
                            provisionSum            = rec['provisionSum'],
                            taxationType            = rec['taxationType'],
                    )
                    receipt.save()


                    # Записать номенклатуру покупок
                    lst = []
                    for i,ser in df.iterrows():
                        rec = ser.to_dict()
                        obj = FNS_Receipt_Product(
                            receipt_id   = receipt.id,
                            name         = rec['name'],        # object 
                            price        = rec['price']/100,   # int64  
                            amount       = rec['sum']/100,     # int64  
                            quantity     = rec['quantity'],    # float64
                            paymentType  = rec.pop('paymentType', ''), # int64  
                            productType  = rec.pop('productType', ''), # int64  
                            nds          = '' #rec['nds'],         # int64  
                        )
                        lst.append(obj)
                    FNS_Receipt_Product.bulk_create(lst, batch_size=999)
                    
                except Exception as e:                    
                    print('При попытке сохранить в базу данных загруженные данные возникла ошибка')
                    print(e)
                    transaction.rollback()
                    raise
    except:
        return 'fail'        
    return 'Ok'      
    
path = 'data_input/my_tickets_json'
path_complete = path + '/complete'
lst_files = os.listdir(path, )
for fn in lst_files:
    if fn.endswith(".json"):
        fn = path + '/' + fn
        res = save_json_to_database(fn)        
        print(res + ': ' + fn)

        if res =='Ok' or res == 'duplicate':
            os.replace(
                    path + '/' + fn, 
                    path + '/completed' + '/' + fn)

# save_json_to_database('data_input/my_tickets_json/20_04_2022_03_31_526596891682004009171.json')


duplicate
duplicate


In [40]:
db.close()

True

In [39]:
counts = {
    'Ok': 0,
    'duplicate': 0,
    'fail': 0
}

mess = str(counts)[1:-1].replace(', ', '\n')
print(mess)

'Ok': 0
'duplicate': 0
'fail': 0


In [69]:
import json

# file_name = 'data_input/tests/test_01.json'
file_name = 'data_input/tests/test_02_main.json'
data=None

with open(file_name, encoding='utf-8') as json_file:
    data = json.load(json_file) #, cls= encoding='cp1251').

head=data[0]
items = head.pop('items')

# for ind in head:
#     print(f'{ind}: ',  head[ind])
# print('------------')
# print(items)
# for ind in items:
#     print(ind)
